# The Dice Game
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can play a simple dice game. Here are the basic rules:
* You start with 0 points
* Roll three fair six-sided dice
* Now choose one of the following:
 * Stick, accept the values shown. If two or more dice show the same values, then all of them are flipped upside down: 1 becomes 6, 2 becomes 5, 3 becomes 4, and vice versa. The total is then added to your points and this is your final score.
 * OR reroll the dice. You may choose to hold any combination of the dice on the current value shown. Rerolling costs you 1 point – so during the game and perhaps even at the end your score may be negative. You then make this same choice again.

The best possible score for this game is 18 and is achieved by rolling three 1s on the first roll.

The reroll penalty prevents you from rolling forever to get this score. If the value of the current dice is greater than the expected value of rerolling them (accounting for the penalty), then you should stick.

The optimal decision is independent of your current score. It does not matter whether it is your first roll with a current score of 0, or your twentieth roll with a current score of -19 (in which case a positive end score is impossible), in either of these cases if you roll three 6s (which, if you stick, will only add 3 points) then you still expect to get a *better* end score by rerolling and taking the penalty. Almost any other roll will beat it, so it's still the right choice to maximise your score.

It is pretty obvious that you should stick on three 1s, and reroll on three 6s. Should you hold any of the 6s when you reroll? What about other values? What should you do if the dice come up 3, 4, 5?

We do not know what numbers will come up when we roll, but we do know exactly what the probability of any given roll is. This is the point of the probabilistic reasoning section of the unit; if we can model the true probabilities then we can mathematically calculate the optimal policy. Not all real world situations use dice, but these techniques work well even if we can only estimate the true probabilities.

### Play The Game
You can play the game in the following cell. Change the `SKIP_GAME` constant to `False` to enable this cell. 
<br> *Make sure to change it back to `True` before submitting.*

In [ ]:
SKIP_GAME = True
if not SKIP_GAME:
    %run dice_game.py

### Submission Requirements
The code supports playing this game with many possible modifications – you can change the number of dice, the values on the dice, or even make biased (weighted) dice that are more likely to roll certain values. More on this later.

For this assignment you will need to submit:
1. The implementation for an agent which can play the game – this notebook
 * You can write this however you like, using the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned depending on how well your agent plays the game (potentially with modifications)
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests and skeleton code included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section
 
### Choice of Algorithm
You can take any approach you like to write your agent. We have provided two examples for you, though these agents do not perform very well, and would not pass the assignment, they should help you structure your code.

We have covered *value iteration* in the unit, and this technique will work well here to produce a strategy for the game, which your agent can then follow. It is up to you to work out the parameters that will be suit the value iteration algorithm to maximise your score.

However, there are many other possible options. Simply calculating the expected value of a single roll will produce a much stronger strategy than playing randomly. You could also look up various other approaches that can be applied to Markov decision processes, such as policy iteration.

To get a high grade on this assignment will require a particularly efficient implementation value iteration with intelligent choice of parameters, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

Note that you can write your agent to support modified versions of the game, which is also demonstrated below. For submissions that support this, the tests will include modified versions of the game, some where the game is *more obvious*, and some where it is *less obvious* than the default rules. Getting high marks requires supporting this feature. For those who are struggling to write a good agent, supporting this feature will result in additional tests which are more forgiving, so this might be an attractive option.

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Dice Game Class
A class called `DiceGame` is provided within `dice_game.py` for you to use in your solution. Here is a demonstration of its features.

When you create a DiceGame object, by default you will get the rules as stated above. 
```python
game = DiceGame()
```
Creates a game with 3 normal 6-sided dice. 

You may wish to modify the game mechanics, and you can do this using the other constructor arguments, for example:
```python
game = DiceGame(dice=4, sides=3, values=[1, 2, 6], bias=[0.1, 0.1, 0.8], penalty=2)
```
will create a game where you roll 4 dice, each with 3 sides, labelled 1, 2, and 6, where each die is far more likely to roll a 6 than they are to roll a 1 or a 2, and furthermore the penalty for rerolling is now 2 points instead of 1. *Note: this does not necessarily result in an interesting game.*

In games with unusual values or sides (3-sided dice are unusual without trying to turn them upside down), when there are duplicates, `value[i]` becomes `value[-i]`. With odd-sided dice, the middle value will flip onto itself.

Once created, the `DiceGame` object can be run in two different modes, *simulation* and *analysis*. It is likely that you will mostly use [*analysis* mode](#Analysis-Mode) to derive your agent's behaviour, but either way some understanding of simulation mode might be useful.

### Simulation Mode
The object provides the methods required to simulate playing the game. This might be useful if you want to test your agent, or you just want to try playing the game yourself, as we did in the cell earlier. The current dice values are found by calling `get_dice_state()`, they will always be listed in ascending order. 

In [ ]:
from dice_game import DiceGame
import numpy as np

# setting a seed for the random number generator gives repeatable results, making testing easier!
np.random.seed(111)

game = DiceGame()
game.get_dice_state()

To roll the dice, you call the `roll` method which takes one parameter: a tuple representing which dice you want to hold, numbered from zero. We rolled (2, 3, 4). Suppose we want to hold the 2, we would pass the tuple `(0,)` into the `roll` method (note we need to include the comma so that Python knows this is a tuple).

The `roll` method returns a tuple containing: the reward for this action, the new state, and whether the game is over.

In [ ]:
reward, new_state, game_over = game.roll((0,))
print(reward)
print(new_state)
print(game_over)

Now suppose we are happy and wish to stick to get our final score. We can call the `roll` method and supply a tuple containing all three dice.

In [ ]:
reward, new_state, game_over = game.roll((0, 1, 2))
print(reward)
print(new_state)
print(game_over)

Remember that the return value is just the reward for the action, in this case 15. To get our final score we can inspect `game.score`. We rerolled once, so expect to get a score of 14.

In [ ]:
print(game.score)

And to play again we can call `game.reset()` which returns the new starting dice.

In [ ]:
game.reset()

### Analysis Mode
In analysis mode, you are not playing the game, but asking the object for *all possible outcomes of certain actions*.

First of all, it is useful to know that all the possible states and all the possible actions are stored inside the object. Try changing the game mechanics on the first line (e.g. add `dice=2` to the constructor) and see how the other information is updated.

In [ ]:
game = DiceGame()
print(f"The first 5 of {len(game.states)} possible dice rolls are: {game.states[0:5]}")
print(f"The possible actions on any given turn are: {game.actions}")

Finally, the most important method is `get_next_states(action, dice_state)`. This allows you to get all the possible resulting states for any given state and action.

Earlier we had the roll of `(2, 3, 4)` and decided to hold the 2. The game can calculate all possible outcomes for us, and crucially will also give us the probability of each state occurring.

In [ ]:
game = DiceGame()
states, game_over, reward, probabilities = game.get_next_states((0,), (2, 3, 4))
for state, probability in zip(states, probabilities):
    print(f"Would get roll of {state} with probability {probability}")

The method also works consistently when all dice are held, reporting that this action would cause the game to be over and giving the reward. Note that the list of states returned contains the value `None`. This is to denote that the game has entered a terminal state – no further actions would be allowed. The game does not return the final dice here, because that is another valid state (from which there would still be actions available). Also, the `reward` value is not the same as the final `score` of any given game, because it does not include any possible previous penalties for rerolling.

In [ ]:
states, game_over, reward, probabilities = game.get_next_states((0, 1, 2), (2, 2, 5))
print(states)
print(game_over)
print(reward)

## Part One
You should write all of your code for your dice game agent below this cell.

Let's start with some example agents, so you can see the format we will use. In the cell below are two agents which do not play particularly well. One always holds immediately, the other will keep re-rolling all dice until they get the best possible dice (`(1, 1, 1)` or `(1, 1, 6)`), ignoring the massive penalty this will incur from re-rolling. Neither of them is considering the probabilities involved in the game.

There is also a function which will run the game with an instance of a given agent. When you run the cell, it will simulate a game with each agent.

In [ ]:
from abc import ABC, abstractmethod


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass


class AlwaysHoldAgent(DiceGameAgent):
    def play(self, state):
        return (0, 1, 2)


class PerfectionistAgent(DiceGameAgent):
    def play(self, state):
        if state == (1, 1, 1) or state == (1, 1, 6):
            return (0, 1, 2)
        else:
            return ()
        
        
def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    while not game_over:
        action = agent.play(state)
        actions += 1
        
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score


if __name__ == "__main__":
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    np.random.seed(1)
    
    game = DiceGame()
    
    agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")
    
    agent2 = PerfectionistAgent(game)
    play_game_with_agent(agent2, game, verbose=True)

In the cell below, write the logic for your own agent. To do this, you need to write a class named `MyAgent` which is a subclass of `DiceGameAgent`, which overrides the `play(self, state)` method. There is some skeleton code in the cell below to help you get started. This is the code which will be tested.

In [84]:
class MyAgent(DiceGameAgent):
    
    def __init__(self, game, run_iterations=True, theta=0.001, gamma=0.95):
        super().__init__(game)
        self._theta_squared = theta ** 2
        self._deltas_squared = []
        self._gamma = gamma
        self._state_action_value = {state: 0 for state in self.game.states}
        self._state_best_action = {state: () for state in self.game.states}
        self._next_states_dict = {state: {} for state in self.game.states}
        self._get_all_next_states()
        self._iterations = 0
        if run_iterations:
            self._iterate_until_minimal_delta()

    def play(self, state):
        return self._state_best_action[state]

    def _iterate_until_minimal_delta(self):
        self._iterate_all_states()
        while max(self._deltas_squared) > self._theta_squared:
            self._iterate_all_states()

    def _iterate_all_states(self):
        self._deltas_squared = []
        for state in self._state_action_value:
            self._update_state_best_action(state)
        self._iterations += 1

    def _update_state_best_action(self, state):
        action_values = [self._calculate_action_value(action=action, state=state) for action in self.game.actions]
        best_action_value = max(action_values)
        self._deltas_squared.append((self._state_action_value[state] - best_action_value) ** 2)
        self._state_action_value[state] = best_action_value
        self._state_best_action[state] = self.game.actions[action_values.index(best_action_value)]

    def _calculate_action_value(self, action, state):
        states, game_over, reward, probabilities = self._next_states_dict[state][action]
        if game_over:
            return self.game.final_scores[state]
        expected_value = sum([self._state_action_value[s] * p for p, s in zip(probabilities, states)])
        return reward + self._gamma * expected_value

    def _get_all_next_states(self):
        for state in self.game.states:
            for action in self.game.actions:
                self._next_states_dict[state][action] = self.game.get_next_states(action=action,
                                                                                  dice_state=state)


You're free to modify the structure of the code above as you like, but you must make sure your code is compatible with our testing framework, which you can check below.

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
Your agent will be tested in two ways. First it will be tested in actual random games, and the average score will be measured. All students will get the exact same dice rolls, so the best strategies will get the most points. Second, it will be analysed in specific circumstances (i.e. specific dice rolls) to test what it does compared to the optimal strategy.

In addition, if your code supports it, we will repeat these tests with *extended rules*, i.e. not using the default game with three 6-sided fair dice. You can read more about how this affects grading in [this section](#Choice-of-Algorithm). 

The tests run with a timeout to prevent any submissions slowing down the entire grading. On the testing machine, your agent must take less than 30 seconds to construct, and less than 2 seconds to produce each action. For high marks, execution time will also factor into grading (both construction and average per move time).

As we have said before, the best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
Run the following cell to test your code 10 times and print the average score. You should run this test to ensure your code is working correctly. The actual tests which determine your grade will be different and more thorough. 

To enable the tests, set the constant `SKIP_TESTS` to `False`.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [86]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time

    total_score = 0
    total_time = 0
    n = 10

    np.random.seed()

    print("Testing basic rules.")
    print()

    game = DiceGame()

    start_time = time.process_time()
    test_agent = MyAgent(game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Total time: {total_time:.4f} seconds")

Testing basic rules.

Game 0 score: 11
Game 1 score: 13
Game 2 score: 10
Game 3 score: 11
Game 4 score: 13
Game 5 score: 11
Game 6 score: 12
Game 7 score: 17
Game 8 score: 14
Game 9 score: 12

Average score: 12.4
Total time: 1.0792 seconds


#### Opt-In For Extended Rules
If you wish your code to be tested with rules other than the defaults, set the constant `TEST_EXTENDED_RULES` to `True` on the next line. Another test will be performed to check your code still works. If you get an error, you are likely not supporting the extended rules properly.

Refer to [this section](#Choice-of-Algorithm) to understand more about how extended rules factor into your possible grade.

**Note:** you need to have `SKIP_TESTS` set to `False` in the cell above (and run it!) to enable the tests below. The value of `TEST_EXTENDED_RULES` *will* be used in the grading tests.

In [ ]:
TEST_EXTENDED_RULES = False

if not SKIP_TESTS and TEST_EXTENDED_RULES:
    total_score = 0
    total_time = 0
    n = 10

    print("Testing extended rules – two three-sided dice.")
    print()

    game = DiceGame(dice=2, sides=3)

    start_time = time.process_time()
    test_agent = MyAgent(game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Average time: {total_time/n:.5f} seconds")

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
import sys
import pathlib

fail = False;

if not SKIP_GAME:
    fail = True;
    print("You must set the SKIP_GAME constant to True in the earlier cell.")

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the earlier cell.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./dicegame.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named dicegame.ipynb")
    
if "MyAgent" not in dir():
    fail = True;
    print("You must include a class called MyAgent as defined above.")
else:    
    game = DiceGame()
    agent = MyAgent(game)
    action = agent.play((1, 1, 1))

    if action not in game.actions:
        print("Warning:")
        print("Your agent does not seem to produce a valid action with the default rules.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
        
    if TEST_EXTENDED_RULES:
        print("Info: TEST_EXTENDED_RULES is ON")
        game = DiceGame(dice=2, sides=8)
        agent = MyAgent(game)
        try:
            action = agent.play((7, 8))
        except:
            action = None

        if action not in game.actions:
            fail = True
            print("Your agent does not produce a valid action with the extended rules.")
            print("Turn off TEST_EXTENDED_RULES if you cannot fix this error.")
    else:
        print("Info: TEST_EXTENDED_RULES is OFF (extended rules will not be tested)")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print()
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

In [ ]:
# This is a TEST CELL. Do not delete or change.